# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [4]:
X = boston_features[['B', 'DIS', 'LSTAT']]
y = pd.DataFrame(boston.target, columns=['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [5]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [8]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

404
102
404
102


Fit the model and apply the model to the make test set predictions

In [11]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_hat_test = linreg.predict(X_test)
train_err = mean_squared_error(y_train, y_hat_train)
test_err = mean_squared_error(y_test, y_hat_test)

Calculate the residuals and the mean squared error

In [13]:
from sklearn.metrics import mean_squared_error
residuals = y_hat_test - y_test
test_mse = mean_squared_error(y_test, y_hat_test)

#print(residuals)
print(test_mse)

21.015890568240792


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [25]:
def kfolds(data, k):
    # Force data as pandas dataframe
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k
    folds = []
    start_observations = 0
    for fold_n in range(1, k+1):
        if fold_n <= leftovers:
            fold =  data.iloc[start_observations : start_observations + fold_size+1] 
            folds.append(fold)
            start_observations +=  fold_size + 1
        else:
            fold =  data.iloc[start_observations : start_observations+fold_size] 
            folds.append(fold)
            start_observations +=  fold_size
    # add 1 to fold size to account for leftovers           
    return folds

### Apply it to the Boston Housing Data

In [19]:
# Make sure to concatenate the data again
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [26]:
bos_folds = kfolds(bos_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [27]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bos_folds) if i!=n])
    test = bos_folds[n]
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print(train_errs)
print(test_errs)

[target    27.416804
dtype: float64, target    27.119089
dtype: float64, target    20.510318
dtype: float64, target    23.111771
dtype: float64, target    27.526491
dtype: float64]
[target    19.514664
dtype: float64, target    19.33714
dtype: float64, target    49.270498
dtype: float64, target    37.651359
dtype: float64, target    17.91502
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [31]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
cv_5_results  = cross_val_score(linreg, X, y, cv=5,  scoring='neg_mean_squared_error')

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [34]:
print(cv_5_results)
print(np.mean(cv_5_results))

[-19.51466367 -19.33713985 -49.27049788 -37.65135885 -17.91502022]
-28.737736096449662


##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!